In [0]:
gedcom_path = "file:/Workspace/Users/edballesq@gmail.com/genealogy/uploads/Genealogy of Ed Ball.ged"

rows = []

with open(gedcom_path.replace("file:", ""), "r", encoding="utf-8", errors="ignore") as f:
    for i, line in enumerate(f):
        line = line.rstrip("\n")
        parts = line.split(" ", 2)

        level = int(parts[0])
        xref_id = None
        tag = None
        value = None

        if len(parts) == 2:
            tag = parts[1]
        elif len(parts) == 3:
            if parts[1].startswith("@") and parts[1].endswith("@"):
                xref_id = parts[1]
                tag = parts[2]
            else:
                tag = parts[1]
                value = parts[2]

        rows.append((i, level, xref_id, tag, value))

df = spark.createDataFrame(
    rows,
    ["line_no", "level", "xref_id", "tag", "value"]
)

display(df.limit(50))


In [0]:
from pyspark.sql.functions import lit

(
    df
    .withColumn("source_file", lit("family_tree_2025-12-26.ged"))
    .write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("genealogy.bronze_gedcom")
)


In [0]:
%sql
SELECT COUNT(*) FROM genealogy.bronze_gedcom;


In [0]:
%sql
CREATE OR REPLACE VIEW genealogy.bronze_gedcom_with_record AS
SELECT
  *,
  last_value(
    xref_id,
    true
  ) OVER (
    ORDER BY line_no
    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
  ) AS record_xref
FROM genealogy.bronze_gedcom;


In [0]:
%sql
SELECT
  record_xref,
  COUNT(*) AS lines
FROM genealogy.bronze_gedcom_with_record
GROUP BY record_xref
ORDER BY lines DESC
LIMIT 10;
